In [ ]:
import zipfile
import numpy as np
import cv2

# Import dataset gambar pemandangan
def load_dataset(path):
    data = []
    with zipfile.ZipFile(path, 'r') as zip_ref:
        for filename in zip_ref.namelist():
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                with zip_ref.open(filename) as img_file:
                    img = cv2.imdecode(np.frombuffer(img_file.read(), np.uint8), -1)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    label = filename.split("/")[-2].lower()  # Ekstrak label dan ubah ke lowercase
                    label_index = hash(label) % 6  # Ubah label ke bilangan bulat positif
                    data.append((img, label_index))
    return data

# Fitur gambar
def extract_features(img):
    return img.flatten()

# Fungsi untuk menentukan kelas gambar
def predict_class(features, pheromones):
    probs = []
    for label in range(6):
        probs.append(pheromones[label] * np.exp(-np.sum((features - features_mean[label])**2 / (2 * sigma[label]**2))))
    return np.argmax(probs)

# Ant Colony Optimization
def aco(data, pheromones, alpha, beta, q0, max_iter):
    for iter in range(max_iter):
        for img, label in data:
            features = extract_features(img)
            next_label = predict_class(features, pheromones)
            label_index = int(label)
    return pheromones

# Main
path = "/content/Archive.zip"
data = load_dataset(path)
features_mean = np.array([np.mean(extract_features(img)) for img, _ in data])
sigma = np.array([np.std(extract_features(img)) for img, _ in data])
pheromones = np.ones((6,))

# Pelatihan
pheromones = aco(data, pheromones, alpha=0.5, beta=1.0, q0=0.1, max_iter=100)

# Evaluasi
correct = 0
for img, label in data:
    predicted_label = predict_class(extract_features(img), pheromones)
    if predicted_label == label:
        correct += 1
accuracy = correct / len(data) * 100

print("Akurasi:", accuracy, "%")

KeyboardInterrupt: ignored